# 語義內核工具使用範例

本文檔提供了基於語義內核的工具的代碼概覽和說明，該工具與 Azure AI Search 集成，用於檢索增強生成（RAG）。此範例展示了如何構建一個 AI 代理，從 Azure AI Search 索引中檢索旅遊文件，使用語義搜索結果增強用戶查詢，並串流詳細的旅遊建議。


### 匯入套件
以下程式碼匯入所需的套件：


In [ ]:
import json
import os

from typing import Annotated

from IPython.display import display, HTML

from dotenv import load_dotenv

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent,FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

### 建立語意核心和 AI 服務

語意核心實例會被建立並配置為非同步的 OpenAI 聊天完成服務。該服務會被添加到核心中，用於生成回應。


In [ ]:
load_dotenv()
# Initialize the asynchronous OpenAI client
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)

# Create the OpenAI Chat Completion Service
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

### 定義提示插件

PromptPlugin 是一個原生插件，定義了一個函數，用於利用檢索上下文構建增強提示


In [ ]:
class SearchPlugin:

    def __init__(self, search_client: SearchClient):
        self.search_client = search_client

    @kernel_function(
        name="build_augmented_prompt",
        description="Build an augmented prompt using retrieval context or function results.",
    )
    def build_augmented_prompt(self, query: str, retrieval_context: str) -> str:
        return (
            f"Retrieved Context:\n{retrieval_context}\n\n"
            f"User Query: {query}\n\n"
            "First review the retrieved context, if this does not answer the query, try calling an available plugin functions that might give you an answer. If no context is available, say so."
        )
    
    @kernel_function(
        name="retrieve_documents",
        description="Retrieve documents from the Azure Search service.",
    )
    def get_retrieval_context(self, query: str) -> str:
        results = self.search_client.search(query)
        context_strings = []
        for result in results:
            context_strings.append(f"Document: {result['content']}")
        return "\n\n".join(context_strings) if context_strings else "No results found"

In [ ]:
class WeatherInfoPlugin:
    """A Plugin that provides the average temperature for a travel destination."""

    def __init__(self):
        # Dictionary of destinations and their average temperatures
        self.destination_temperatures = {
            "maldives": "82°F (28°C)",
            "swiss alps": "45°F (7°C)",
            "african safaris": "75°F (24°C)"
        }

    @kernel_function(description="Get the average temperature for a specific travel destination.")
    def get_destination_temperature(self, destination: str) -> Annotated[str, "Returns the average temperature for the destination."]:
        """Get the average temperature for a travel destination."""
        # Normalize the input destination (lowercase)
        normalized_destination = destination.lower()

        # Look up the temperature for the destination
        if normalized_destination in self.destination_temperatures:
            return f"The average temperature in {destination} is {self.destination_temperatures[normalized_destination]}."
        else:
            return f"Sorry, I don't have temperature information for {destination}. Available destinations are: Maldives, Swiss Alps, and African safaris."

## 向量資料庫初始化

我們使用持久性存儲初始化 Azure AI Search，並添加增強的範例文件。Azure AI Search 將用於存儲和檢索文件，以提供生成準確回應的背景資訊。


In [ ]:
# Initialize Azure AI Search with persistent storage
search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = "travel-documents"

search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_api_key)
)

index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=AzureKeyCredential(search_api_key)
)

# Define the index schema
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="content", type=SearchFieldDataType.String)
]

index = SearchIndex(name=index_name, fields=fields)

# Check if index already exists if not, create it
try:
    existing_index = index_client.get_index(index_name)
    print(f"Index '{index_name}' already exists, using the existing index.")
except Exception:
    # Create the index if it doesn't exist
    print(f"Creating new index '{index_name}'...")
    index_client.create_index(index)


# Enhanced sample documents
documents = [
    {"id": "1", "content": "Contoso Travel offers luxury vacation packages to exotic destinations worldwide."},
    {"id": "2", "content": "Our premium travel services include personalized itinerary planning and 24/7 concierge support."},
    {"id": "3", "content": "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage."},
    {"id": "4", "content": "Popular destinations include the Maldives, Swiss Alps, and African safaris."},
    {"id": "5", "content": "Contoso Travel provides exclusive access to boutique hotels and private guided tours."}
]

# Add documents to the index
search_client.upload_documents(documents)

In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[SearchPlugin(search_client=search_client), WeatherInfoPlugin()],
    name="TravelAgent",
    instructions="Answer travel queries using the provided tools and context. If context is provided, do not say 'I have no context for that.'",
)

### 使用串流調用運行代理

主要的非同步循環會為對話創建一個執行緒，並針對每次用戶輸入，讓代理能夠看到檢索的上下文。同時也會加入用戶的訊息，然後使用串流方式調用代理。輸出會在串流過程中即時打印出來。


In [ ]:
async def main():
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Can you explain Contoso's travel insurance coverage?",
        "What is the average temperature of the Maldives?",
        "What is a good cold destination offered by Contoso and what is it average temperature?",
    ]

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


# 重要規則

在開始之前，請確保你已經熟悉以下規則：

1. 確保所有的程式碼片段（例如 ```text
User: 
Can you explain Contoso's travel insurance coverage?

Function Calls (click to expand)

Calling function: retrieve_documents({"query": "Contoso travel insurance coverage"})

Function Result:

Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.

Document: Contoso Travel provides exclusive access to boutique hotels and private guided tours.

Document: Our premium travel services include personalized itinerary planning and 24/7 concierge support.

TravelAgent:

Contoso's travel insurance coverage includes the following:

1. **Medical Emergencies**: Coverage for unforeseen medical issues that may arise while traveling.
2. **Trip Cancellations**: Protection in case you need to cancel your trip for covered reasons.
3. **Lost Baggage**: Compensation for baggage that is lost during your trip.

If you need more specific details about the policy, it would be best to contact Contoso directly or refer to their official documentation.
```）保持原樣，並且不要進行任何翻譯。
2. 請勿翻譯變數名稱、函數名稱、類別名稱或任何佔位符（例如 @@INLINE_CODE_0@@）。
3. 保持所有原始的 Markdown 格式，例如標題、列表、連結等。
4. 如果文件中包含註解，請務必翻譯這些註解。
5. 請勿翻譯以下標籤：[!NOTE]、[!WARNING]、[!TIP]、[!IMPORTANT]、[!CAUTION]。
6. 如果文件中包含 URL 或路徑，請保持原樣，不要進行翻譯。

## 範例

以下是一個簡單的範例，展示如何正確處理翻譯：

```markdown
# 範例標題

這是一段描述如何使用 @@INLINE_CODE_0@@ 的文字。

[!NOTE]
這是一個重要的提示，請務必仔細閱讀。

## 下一步

請參考以下連結以獲取更多資訊：
- [官方文件](https://example.com)
- [GitHub 儲存庫](https://github.com/example)
```

在這個範例中：
- 標題 "# 範例標題" 被翻譯為中文。
- 描述文字被翻譯為中文，但 @@INLINE_CODE_0@@ 保持不變。
- [!NOTE] 標籤未被翻譯。
- URL 保持原樣。

## 注意事項

在翻譯過程中，請務必保持專業性，並確保翻譯後的內容自然流暢，符合中文（繁體，香港）的語言習慣。



---

**免責聲明**：  
本文件已使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。原始語言的文件應被視為權威來源。對於重要信息，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋概不負責。
